In [1]:
import mydemosaicking_header as mh
from mydensenet import DenseNet 

#read data
train_gt_path = './data/train/gt/'
train_train_path = './data/train/train/'
valid_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/gt/'
valid_valid_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/valid/'
McM_test_path = './data/test/McM_test/'
McM_gt_path = './data/test/McM_gt/'
McM_test_img = []
McM_gt_img = []
train_gt_patch = []
train_train_patch = []
valid_gt_img = []
valid_valid_img = []
#rgb = ['r','g','b']

#for i in rgb:
#    gt_patch = '_gt_patch_all.h5'
#    train_patch = '_train_patch_all.h5'
#    gt_filename = i+gt_patch
#    train_filename = i+train_patch
#    temp = mh.load_h5(train_gt_path, gt_filename, 1)
#    train_gt_patch.append(temp)
#    temp = mh.load_h5(train_train_path, train_filename, 1)
#    train_train_patch.append(temp)
##    for j in range(1,101):
##        valid_img = '_valid_img_'
##        gt_img = '_gt_img_'
##        h5 = '.h5'
##        valid_filename = str(j) + valid_img + i + h5
##        gt_filename = str(j) + gt_img + i + h5
##        temp = mh.load_h5(valid_valid_path, valid_filename, j)
##        valid_valid_img.append(temp)
##        temp = mh.load_h5(valid_gt_path, gt_filename, j)
##        valid_gt_img.append(temp)
#    for j in range(1,19):
#        test_img = '_test_img_'
#        gt_img = '_gt_img_'
#        h5 = '.h5'
#        test_filename = str(j) + test_img + i + h5
#        gt_filename = str(j) + gt_img + i + h5
#        temp = mh.load_h5(McM_test_path, test_filename, j)
#        McM_test_img.append(temp)
#        temp = mh.load_h5(McM_gt_path, gt_filename, j)
#        McM_gt_img.append(temp)    

#RGB ver.
gt_filename = 'gt_patch_all.h5'
train_filename = 'train_patch_all.h5'
temp = mh.load_h5_RGB(train_gt_path, gt_filename, 1)
train_gt_patch.append(temp)
temp = mh.load_h5_RGB(train_train_path, train_filename, 1)
train_train_patch.append(temp)

for j in range(1,19):
    test_img = '_test_img'
    gt_img = '_gt_img'
    h5 = '.h5'
    test_filename = str(j) + test_img + h5
    gt_filename = str(j) + gt_img + h5
    temp = mh.load_h5_RGB(McM_test_path, test_filename, j)
    McM_test_img.append(temp)
    temp = mh.load_h5_RGB(McM_gt_path, gt_filename, j)
    McM_gt_img.append(temp)   

/home/pbj/anaconda3/envs/tf15/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
BATCH_SIZE = 64
EPOCHS = 200
w_path = './checkpoints/weights-improvement-100-36.81.hdf5'

model = DenseNet(nb_dense_block=10, growth_rate=32, nb_filter=64, reduction=0.0, dropout_rate=0.0, weights_path=None)

adam = mh.Adam(lr=0.0001)
model.compile(adam, loss='mse', metrics=[mh.PSNR, "accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 4 0                                            
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 1 60          data[0][0]                       
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 1 0           conv0[0][0]                      
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 6976        leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
leaky_re_l

In [ ]:
#r_filepath="./checkpoints/red/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
#r_checkpoint = mh.ModelCheckpoint(r_filepath, monitor=mh.PSNR, verbose=1, mode='max')
#r_callbacks_list = [r_checkpoint]

#g_filepath="./checkpoints/green/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
#g_checkpoint = mh.ModelCheckpoint(g_filepath, monitor=mh.PSNR, verbose=1, mode='max')
#g_callbacks_list = [g_checkpoint]

#b_filepath="./checkpoints/blue/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
#b_checkpoint = mh.ModelCheckpoint(b_filepath, monitor=mh.PSNR, verbose=1, mode='max')
#b_callbacks_list = [b_checkpoint]

rgb_filepath="./checkpoints/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
rgb_checkpoint = mh.ModelCheckpoint(rgb_filepath, monitor=mh.PSNR, verbose=1, mode='max')
rgb_callbacks_list = [rgb_checkpoint]

model.fit_generator(mh.train_image_gen_RGB(train_train_patch, train_gt_patch, BATCH_SIZE), steps_per_epoch=522939 // BATCH_SIZE,  \
					validation_data=mh.valid_image_gen_RGB(McM_test_img, McM_gt_img, 18), validation_steps=18, \
					epochs=EPOCHS, workers=8, callbacks=rgb_callbacks_list)

print("Done training!!!")

print("Saving the final model ...")

#model.save('vdsr_model.h5')  # creates a HDF5 file 
del model  # deletes the existing model

Epoch 1/200
8170/8170 [==============================] - 1457s 178ms/step - loss: 0.0033 - PSNR: 26.7180 - acc: 0.8400 - val_loss: 0.0014 - val_PSNR: 29.4001 - val_acc: 0.8808

Epoch 00001: saving model to ./checkpoints/weights-improvement-01-26.72.hdf5
Epoch 2/200
8170/8170 [==============================] - 1453s 178ms/step - loss: 0.0013 - PSNR: 28.8826 - acc: 0.8829 - val_loss: 9.0377e-04 - val_PSNR: 31.2623 - val_acc: 0.8880

Epoch 00002: saving model to ./checkpoints/weights-improvement-02-28.88.hdf5
Epoch 3/200
4148/8170 [==============>...............] - ETA: 11:54 - loss: 0.0011 - PSNR: 29.8318 - acc: 0.8935